In [ ]:
pip install pydicom numpy matplotlib scipy scikit-image pyvista panel

In [ ]:
import pydicom
import numpy as np
import matplotlib.pyplot as plt
import os
import glob
from scipy.ndimage import zoom # For resampling
from skimage.measure import marching_cubes # For surface extraction
import pyvista as pv
pv.set_jupyter_backend('panel') # Or 'ipygany', 'pythreejs', etc. if preferred/installed

In [ ]:
# --- 1. Load and Sort DICOM Slices ---
def load_scan(path):
    """
    Loads DICOM files from a specified directory, sorts them by slice position,
    and extracts relevant metadata.

    Args:
        path (str): Path to the directory containing DICOM files.

    Returns:
        tuple: (list of pydicom datasets, tuple of pixel spacing (x, y), float slice thickness)
               Returns (None, None, None) if loading fails or no DICOMs found.
    """
    slices = []
    dicom_files = glob.glob(os.path.join(path, '*.dcm'))
    if not dicom_files:
        print(f"Error: No .dcm files found in {path}")
        return None, None, None

    for f in dicom_files:
        try:
            ds = pydicom.dcmread(f)
            # Check if it's an image slice with position info
            if hasattr(ds, 'ImagePositionPatient') and hasattr(ds, 'PixelData'):
                slices.append(ds)
            # else:
            #     print(f"Skipping non-image or incomplete file: {f}")
        except Exception as e:
            print(f"Warning: Could not read file {f}: {e}")

    if not slices:
        print(f"Error: No valid DICOM image slices found in {path}")
        return None, None, None

    # Sort slices based on the Z coordinate of ImagePositionPatient
    # ImagePositionPatient: (x, y, z) position of the top-left corner
    # of the image in mm in the patient coordinate system.
    try:
        slices.sort(key=lambda x: float(x.ImagePositionPatient[2]))
    except AttributeError:
        print("Error: Could not sort slices. 'ImagePositionPatient' missing or invalid in some files.")
        # Fallback: Try sorting by SliceLocation if available
        try:
            print("Attempting fallback sort using 'SliceLocation'.")
            slices.sort(key=lambda x: float(x.SliceLocation))
        except AttributeError:
            print("Error: Fallback sort failed. 'SliceLocation' also missing or invalid.")
            return None, None, None
        except Exception as e:
             print(f"Error during fallback sort: {e}")
             return None, None, None
    except Exception as e:
         print(f"Error during primary sort: {e}")
         return None, None, None


    # --- Extract Metadata for Spacing ---
    try:
        # Pixel Spacing (x, y) in mm
        pixel_spacing = slices[0].PixelSpacing # [row spacing, column spacing]
        pixel_spacing = (float(pixel_spacing[1]), float(pixel_spacing[0])) # (x, y) format

        # Slice Thickness vs. Spacing Between Slices
        # SliceThickness: Nominal thickness of the slice in mm.
        # SpacingBetweenSlices: Optional tag, distance between centers of slices. More reliable if present.
        # If SpacingBetweenSlices is not present, calculate from ImagePositionPatient[2] differences.
        if len(slices) > 1:
            slice_thickness = abs(slices[1].ImagePositionPatient[2] - slices[0].ImagePositionPatient[2])
        else:
            # Fallback to SliceThickness if only one slice or calculation fails
            slice_thickness = float(slices[0].SliceThickness) if hasattr(slices[0], 'SliceThickness') else 1.0 # Default to 1 if missing

        # Check for SpacingBetweenSlices tag as potentially more accurate
        if hasattr(slices[0], 'SpacingBetweenSlices'):
             spacing_between = float(slices[0].SpacingBetweenSlices)
             # Use it only if it's significantly different and seems valid (e.g., positive)
             if spacing_between > 0 and abs(spacing_between - slice_thickness) > 1e-3 :
                 print(f"Using SpacingBetweenSlices ({spacing_between}mm) instead of calculated difference ({slice_thickness}mm).")
                 slice_thickness = spacing_between

    except Exception as e:
        print(f"Error extracting metadata: {e}")
        # Provide default spacing if metadata is missing/corrupt
        pixel_spacing = (1.0, 1.0)
        slice_thickness = 1.0
        print("Warning: Using default spacing (1.0, 1.0, 1.0)")

    return slices, pixel_spacing, slice_thickness

# --- 2. Convert Slices to a 3D NumPy Array ---
def get_pixels_hu(scans):
    """
    Converts a list of pydicom datasets into a 3D NumPy array, applying
    Rescale Slope/Intercept to convert to Hounsfield Units (HU) if possible.

    Args:
        scans (list): List of sorted pydicom datasets.

    Returns:
        numpy.ndarray: 3D array of pixel data (in HU if possible, otherwise raw pixel values).
                       Shape: (num_slices, height, width)
    """
    image = np.stack([s.pixel_array for s in scans])
    # Convert to float32 for potential scaling
    image = image.astype(np.float32)

    # Apply rescale slope and intercept (common for CT scans to get HU units)
    # Check if metadata is present in the first slice (assuming consistency)
    if hasattr(scans[0], 'RescaleSlope') and hasattr(scans[0], 'RescaleIntercept'):
        slope = float(scans[0].RescaleSlope)
        intercept = float(scans[0].RescaleIntercept)

        # Ensure slope is not zero to avoid division errors
        if slope != 0:
            print(f"Applying Rescale Slope ({slope}) and Intercept ({intercept}) to convert to HU.")
            image = slope * image + intercept
        else:
             print("Warning: Rescale Slope is 0. Applying only intercept.")
             image = image + intercept # Apply intercept even if slope is 0 or 1

    else:
        print("Warning: Rescale Slope/Intercept not found. Returning raw pixel values.")

    return np.array(image, dtype=np.int16) # Convert to int16 for HU range typical use

# --- 3. Resample Volume to Isotropic Spacing (Optional but Recommended) ---
def resample_volume(image, current_spacing, new_spacing=[1.0, 1.0, 1.0]):
    """
    Resamples the 3D volume to a new spacing using spline interpolation.

    Args:
        image (numpy.ndarray): The 3D input image (shape: z, y, x).
        current_spacing (tuple): The current voxel spacing (dz, dy, dx).
        new_spacing (list, optional): The desired isotropic spacing. Defaults to [1.0, 1.0, 1.0].

    Returns:
        numpy.ndarray: The resampled 3D image.
    """
    if not current_spacing or len(current_spacing) != 3:
        print("Error: Invalid current_spacing provided for resampling.")
        return image # Return original image if spacing is invalid

    # Calculate resize factor
    resize_factor = [current_spacing[i] / new_spacing[i] for i in range(3)]
    # Note: scipy.ndimage.zoom expects zoom factors, not target shape.
    # The input array is (z, y, x) and spacing is often given as (z, y, x) or (x, y, z).
    # Ensure consistency! Let's assume current_spacing = (dz, dy, dx) matching image shape.
    print(f"Resampling from spacing {current_spacing} to {new_spacing}...")
    print(f"Resize factors (z, y, x): {resize_factor}")

    # order=3 means cubic spline interpolation
    # mode='nearest' handles boundaries
    new_image = zoom(image, resize_factor, mode='nearest', order=3)
    print(f"New shape: {new_image.shape}")
    return new_image


# --- Main Execution ---
# Replace with the actual path to YOUR directory containing DICOM slices
dicom_dir = 'C:\Users\fedhila\Downloads\tif_slices-20250505T114141Z-001\tif_slices' # <--- IMPORTANT: CHANGE THIS

# 1. Load and Sort
patient_slices, spacing_xy, thickness_z = load_scan(dicom_dir)

if patient_slices:
    # Combine spacing info (assuming z, y, x order for numpy array)
    # Original spacing: (Slice Thickness, Pixel Spacing Y, Pixel Spacing X)
    original_spacing = (thickness_z, spacing_xy[1], spacing_xy[0])
    print(f"Original Voxel Spacing (Z, Y, X): {original_spacing} mm")

    # 2. Get 3D Array (potentially in HU)
    patient_3d = get_pixels_hu(patient_slices)
    print(f"Original 3D Volume Shape (Z, Y, X): {patient_3d.shape}")

    # --- Visualization ---

    # a) Plot a central slice using Matplotlib
    plt.figure(figsize=(8, 8))
    plt.imshow(patient_3d[patient_3d.shape[0] // 2], cmap=plt.cm.gray) # Show middle slice
    plt.title(f'Central Slice (Index: {patient_3d.shape[0] // 2})')
    plt.xlabel(f'X Pixel (Spacing: {original_spacing[2]:.2f} mm)')
    plt.ylabel(f'Y Pixel (Spacing: {original_spacing[1]:.2f} mm)')
    plt.colorbar(label='Pixel Value (HU or Raw)')
    plt.show()

    # b) 3D Visualization using PyVista (more powerful)

    # Option B1: Resample to isotropic voxels (often improves visualization)
    print("\n--- Resampling for Visualization ---")
    target_isotropic_spacing = [1.0, 1.0, 1.0] # Target 1x1x1 mm resolution
    patient_3d_resampled = resample_volume(patient_3d, original_spacing, new_spacing=target_isotropic_spacing)
    resampled_spacing = target_isotropic_spacing # Use the target spacing for the resampled volume
    # Create a PyVista grid for the *resampled* data
    grid_resampled = pv.UniformGrid()
    # PyVista dimensions are (nx, ny, nz) -> (width, height, depth)
    grid_resampled.dimensions = np.array(patient_3d_resampled.shape)[[2, 1, 0]] # Swap Z,Y,X -> X,Y,Z
    grid_resampled.spacing = resampled_spacing[::-1] # PyVista spacing (dx, dy, dz) -> needs reversed spacing
    # Origin can be set if known, otherwise defaults to (0,0,0)
    # grid_resampled.origin = (0.0, 0.0, 0.0) # Example
    grid_resampled.point_data['values'] = patient_3d_resampled.flatten(order='F') # Flatten in Fortran order (x varies fastest)

    # Option B2: Use original anisotropic data (may look stretched/squashed if not resampled)
    # Create a PyVista grid for the *original* data
    grid_original = pv.UniformGrid()
    grid_original.dimensions = np.array(patient_3d.shape)[[2, 1, 0]] # (nx, ny, nz)
    grid_original.spacing = original_spacing[::-1] # (dx, dy, dz)
    grid_original.point_data['values'] = patient_3d.flatten(order='F')


    # --- Choose which grid to visualize ---
    grid_to_visualize = grid_resampled # Recommended
    # grid_to_visualize = grid_original # If you want to see original spacing effects

    print(f"\n--- Visualizing 3D Volume ({'Resampled' if grid_to_visualize is grid_resampled else 'Original'}) ---")
    print(f"Grid Dimensions (X, Y, Z): {grid_to_visualize.dimensions}")
    print(f"Grid Spacing (X, Y, Z): {grid_to_visualize.spacing}")

    # Setup PyVista Plotter
    plotter = pv.Plotter(window_size=[800, 800])
    plotter.background_color = 'white'

    # Method 1: Volume Rendering (shows semi-transparent volume)
    # Adjust clim (value range) and cmap (colormap) as needed
    # You might need to experiment with opacity mapping for good results
    # Common CT colormaps: 'bone', 'coolwarm', 'viridis'
    # Common HU ranges: [-1000, 1000] for general tissue, [200, 1500] for bone
    clim_range = [-800, 1200] # Example range, adjust based on your data
    plotter.add_volume(grid_to_visualize, cmap='bone_r', clim=clim_range,
                       opacity='sigmoid', shade=True) # 'sigmoid' or 'linear' opacity mapping
    plotter.camera_position = 'iso'
    plotter.add_axes()
    print("Showing Volume Rendering...")
    plotter.show(title="Volume Rendering") # Interactive window

    # Method 2: Isosurface Extraction (shows surfaces at a specific value threshold)
    # Useful for visualizing specific structures like bone or organs if HU values are known.
    # Use scikit-image's marching_cubes or PyVista's contour filter.

    # Using PyVista's contour:
    threshold_value = 300 # Example: Threshold for bone (adjust!)
    contours = grid_to_visualize.contour([threshold_value])

    plotter_iso = pv.Plotter(window_size=[800, 800])
    plotter_iso.background_color = 'white'
    plotter_iso.add_mesh(contours, color="tan", opacity=0.8, smooth_shading=True)
    plotter_iso.camera_position = 'iso'
    plotter_iso.add_axes()
    print(f"\nShowing Isosurface at value {threshold_value}...")
    plotter_iso.show(title=f"Isosurface at {threshold_value}")

    # Using scikit-image marching_cubes (requires data prepared differently)
    # Note: marching_cubes expects (Z, Y, X) data and returns verts/faces in that coordinate system.
    # We need to account for voxel spacing manually when plotting.
    # Use the data *before* flattening for PyVista grid
    data_for_mc = patient_3d_resampled if grid_to_visualize is grid_resampled else patient_3d
    spacing_for_mc = resampled_spacing if grid_to_visualize is grid_resampled else original_spacing

    try:
        verts, faces, _, _ = marching_cubes(data_for_mc, level=threshold_value, spacing=spacing_for_mc) # Provide spacing!
        surface_mesh = pv.PolyData(verts, faces=np.insert(faces, 0, 3, axis=1)) # PyVista requires face format [3, v0, v1, v2]

        plotter_mc = pv.Plotter(window_size=[800, 800])
        plotter_mc.background_color = 'white'
        plotter_mc.add_mesh(surface_mesh, color="lightblue", opacity=0.8, smooth_shading=True)
        plotter_mc.camera_position = 'iso'
        plotter_mc.add_axes()
        print(f"\nShowing Marching Cubes Isosurface (skimage) at value {threshold_value}...")
        plotter_mc.show(title=f"Marching Cubes Isosurface (skimage) at {threshold_value}")

    except Exception as e:
        print(f"Could not generate marching cubes surface: {e}")


else:
    print("Could not load DICOM data. Exiting.")